## Data Cleaning and Regression Analysis

#### In this file, I show the data cleaning and regression analysis of the heterogeneous effects of Medicaid expansion on SNAP participation. I use the NumPy and pandas packages to clean the CPS–ASEC data. I use the linearmodels package to estimate standard two-way fixed-effects models for the non-disabled, non-elderly, Medicaid-eligible population. I further split the sample into parents and non-parents and carry out a similar analysis.

#### Importing Libraries

In [1]:

import pandas as pd
import numpy as np

#### Load Data

In [2]:
file_path = r"C:\Users\chukw\OneDrive\Documentos\Medicaid_and_SNAP\Data and Codes"


In [3]:
DATA_PATH = r"C:\temp\Maindata1.dta"

In [4]:
df = pd.read_stata(DATA_PATH, convert_categoricals=False)


In [5]:
df.shape

df.head(10)

,year,serial,month,hwtfinl,cpsid,asecflag,hflag,asecwth,statefip,county,...,himcaidnw,covergh,coverpi,phinsur,caidly,caidnw,caidpart,state_name,Unemploymentrate,_merge
0,2008,63339,3,NaN,2.008131e+13,1,NaN,1746.74,1,1097,...,NaN,2.0,2.0,2,1,NaN,NaN,AL,5.7,3
1,2008,63216,3,NaN,2.006121e+13,1,NaN,1327.73,1,0,...,NaN,1.0,1.0,1,2,NaN,NaN,AL,5.7,3
2,2008,63206,3,NaN,2.008030e+13,1,NaN,1740.63,1,0,...,NaN,2.0,2.0,2,1,NaN,NaN,AL,5.7,3
3,2008,62395,3,NaN,2.008010e+13,1,NaN,1748.04,1,0,...,NaN,2.0,2.0,2,1,NaN,NaN,AL,5.7,3
4,2008,62402,3,NaN,2.008020e+13,1,NaN,3452.06,1,0,...,NaN,1.0,1.0,1,1,NaN,NaN,AL,5.7,3
5,2008,63070,3,NaN,2.008131e+13,1,NaN,1648.75,1,0,...,NaN,2.0,2.0,2,1,NaN,NaN,AL,5.7,3
6,2008,62863,3,NaN,2.008030e+13,1,NaN,3272.59,1,1073,...,NaN,2.0,2.0,2,1,NaN,NaN,AL,5.7,3
7,2008,62788,3,NaN,2.008131e+13,1,NaN,1329.55,1,0,...,NaN,2.0,2.0,2,1,NaN,NaN,AL,5.7,3
8,2008,63287,3,NaN,2.008030e+13,1,NaN,1433.25,1,1097,...,NaN,2.0,2.0,2,1,NaN,NaN,AL,5.7,3
9,2008,62839,3,NaN,2.008020e+13,1,NaN,3920.40,1,1073,...,NaN,1.0,1.0,1,1,NaN,NaN,AL,5.7,3


In [6]:
[col for col in df.columns if "year" in col]


['year']

In [8]:
[col for col in df.columns if "wt" in col or "weight" in col]


['hwtfinl', 'asecwth', 'wtfinl', 'asecwt']

### Data Cleaning

In [9]:
df = df[df["asecflag"] != 2]


In [10]:
df = df[df["year"] <= 2020]


#### Creating Education Vars

In [11]:
df["educ"] = pd.to_numeric(df["educ"], errors="coerce")
df["Educ"] = np.nan


df.loc[df["educ"] <= 71, "Educ"] = 1
df.loc[df["educ"] == 73, "Educ"] = 2
df.loc[df["educ"].isin([81, 91, 92]), "Educ"] = 3
df.loc[df["educ"] >= 111, "Educ"] = 4


In [12]:
df["Educ"].value_counts(dropna=False)


Educ
1.0    933083
2.0    537493
4.0    534429
3.0    516262
Name: count, dtype: int64

In [13]:

educ_labels = {
    1: "no high sch diploma",
    2: "high sch diploma",
    3: "some college",
    4: "bachelor's degree"
}




In [14]:
df["Educ"].value_counts(dropna=False)


Educ
1.0    933083
2.0    537493
4.0    534429
3.0    516262
Name: count, dtype: int64

#### Education Dummies

In [15]:
df['HSorless'] = (df['educ'] <= 73).astype(int)
df['college'] = (df['educ'] >= 111).astype(int)


#### Female and Race variables

In [16]:

df["sex"] = pd.to_numeric(df["sex"], errors="coerce")
df["female"] = 0
df.loc[df["sex"] == 2, "female"] = 1


In [17]:
df["female"].value_counts(dropna=False)


female
1    1297487
0    1223780
Name: count, dtype: int64

In [60]:
df['white'] = ((df['race'] == 100) & (df['hispan'] == 0)).astype(int)
df['black'] = ((df['race'] == 200) & (df['hispan'] == 0)).astype(int)
df['hispanic'] = (df['hispan'] != 0).astype(int)
df['otherrace'] = ((df['race'] > 200) & (df['hispan'] == 0)).astype(int)


In [19]:
df[['white', 'black', 'hispanic', 'otherrace']].sum()


white        1535285
black         284384
hispanic      467826
otherrace     233772
dtype: int64

#### Create SNAP participation

In [20]:
df['SNAP1'] = df['foodstmp']


#### Marital variable

In [21]:
df["marst"].value_counts(dropna=False)


marst
6    1168716
1     997704
4     186114
5     102083
3      39701
2      26949
Name: count, dtype: int64

In [22]:
df["marst"] = pd.to_numeric(df["marst"], errors="coerce")
df["married"] = np.nan


In [23]:
df.loc[df["marst"].isin([1, 2]), "married"] = 1
df.loc[df["marst"].isin([3, 4, 5, 6]), "married"] = 0


In [24]:
married_labels = {
    1: "married",
    0: "divorced, widowed, separated or single"
}


#### Parent

In [25]:
df['parent'] = 0
df.loc[df['nchild'] > 0, 'parent'] = 1


In [26]:
df["parent"].value_counts(dropna=False)

parent
0    1696397
1     824870
Name: count, dtype: int64

#### Rename unemployment

In [27]:
df = df.rename(columns={'Unemploymentrate': 'ur'})


#### Removing disabled people

In [28]:
df['disable'] = 0
df.loc[df['diffany'] == 2, 'disable'] = 1


In [29]:
df = df[df['disable'] != 1].copy()


#### Drop members of armed forces

In [30]:
df = df[df['empstat'] != 1].copy()


#### Creating Medicaid Eligibility

In [31]:
# Clean family size: replace famsize = . if famsize < 1
df.loc[df['famsize'] < 1, 'famsize'] = np.nan

# Cap family size at 7
df['famsize_cap'] = df['famsize']
df.loc[df['famsize_cap'] > 7, 'famsize_cap'] = 7

# Construct FPL line using HHS poverty guidelines
df['fpl_line'] = np.nan

In [32]:
# Compute fpl_line for a given base and add amount
def fpl_calc(base, add, famsize_cap_series):
    return base + (famsize_cap_series - 1) * add

# Year-specific assignments (matches your Stata replace lines)
df.loc[df['year'] == 2008, 'fpl_line'] = fpl_calc(10800, 3700, df.loc[df['year'] == 2008, 'famsize_cap'])

df.loc[df['year'].isin([2009, 2010, 2011]), 'fpl_line'] = fpl_calc(
    10900, 3800, df.loc[df['year'].isin([2009, 2010, 2011]), 'famsize_cap']
)

df.loc[df['year'] == 2012, 'fpl_line'] = fpl_calc(11170, 3960, df.loc[df['year'] == 2012, 'famsize_cap'])
df.loc[df['year'] == 2013, 'fpl_line'] = fpl_calc(11490, 4020, df.loc[df['year'] == 2013, 'famsize_cap'])
df.loc[df['year'] == 2014, 'fpl_line'] = fpl_calc(11670, 4060, df.loc[df['year'] == 2014, 'famsize_cap'])
df.loc[df['year'] == 2015, 'fpl_line'] = fpl_calc(11770, 4160, df.loc[df['year'] == 2015, 'famsize_cap'])
df.loc[df['year'] == 2016, 'fpl_line'] = fpl_calc(11880, 4160, df.loc[df['year'] == 2016, 'famsize_cap'])
df.loc[df['year'] == 2017, 'fpl_line'] = fpl_calc(12060, 4180, df.loc[df['year'] == 2017, 'famsize_cap'])
df.loc[df['year'] == 2018, 'fpl_line'] = fpl_calc(12140, 4320, df.loc[df['year'] == 2018, 'famsize_cap'])
df.loc[df['year'] == 2019, 'fpl_line'] = fpl_calc(12490, 4420, df.loc[df['year'] == 2019, 'famsize_cap'])
df.loc[df['year'] == 2020, 'fpl_line'] = fpl_calc(12760, 4480, df.loc[df['year'] == 2020, 'famsize_cap'])


In [ ]:
df['fpl_ratio'] = np.nan
mask_ratio = (df['ftotval'] >= 0) & (df['fpl_line'].notna())
df.loc[mask_ratio, 'fpl_ratio'] = df.loc[mask_ratio, 'ftotval'] / df.loc[mask_ratio, 'fpl_line']

# Medicaid income eligibility (ACA adults)
df['medicaid_eligible'] = pd.Series(np.nan, index=df.index)
mask_elig = df['fpl_ratio'].notna()
df.loc[mask_elig, 'medicaid_eligible'] = (df.loc[mask_elig, 'fpl_ratio'] <= 1.38).astype(int)


In [34]:
df['medicaid_eligible'] = df['medicaid_eligible'].astype('Int64')

#### Generate age sample

In [35]:
df['sample1'] = np.nan
df.loc[(df['age'] > 17) & (df['age'] < 65), 'sample1'] = 1

#### Set up for TWFE regression

In [36]:
# Initialize Medicaid expansion indicator
df['medicaid'] = 0

# 2014 January expansion states
states_2014 = [
    4, 5, 6, 8, 9, 10, 11, 15, 17, 19, 21, 24, 25,
    26, 27, 32, 33, 34, 35, 36, 38, 39, 41, 44,
    50, 53, 54
]

df.loc[
    (df['statefip'].isin(states_2014)) & (df['year'] >= 2014),
    'medicaid'
] = 1

# Pennsylvania (Jan 2015)
df.loc[
    (df['statefip'] == 42) & (df['year'] >= 2015),
    'medicaid'
] = 1

# Indiana (Feb 2015)
df.loc[
    (df['statefip'] == 18) & (df['year'] >= 2015),
    'medicaid'
] = 1

# Alaska (Sept 2015)
df.loc[
    (df['statefip'] == 2) & (df['year'] >= 2015),
    'medicaid'
] = 1

# Montana (Jan 2016)
df.loc[
    (df['statefip'] == 30) & (df['year'] >= 2016),
    'medicaid'
] = 1

# Louisiana (July 2016)
df.loc[
    (df['statefip'] == 22) & (df['year'] >= 2016),
    'medicaid'
] = 1

In [37]:
# Initialize Medicaid expansion indicator
df['medic'] = 0

# 2014 January expansion states
states_2014 = [
    4, 5, 6, 8, 9, 10, 11, 15, 17, 19, 21, 24, 25,
    26, 27, 32, 33, 34, 35, 36, 38, 39, 41, 44,
    50, 53, 54,18,2,30,22
]

df.loc[
    (df['statefip'].isin(states_2014)) & (df['year'] >= 2014),
    'medic'
] = 1

Save dataset

In [38]:
df.to_stata("df_analysis.dta", write_index=False)


#### Importing linearmodels

In [39]:
import sys
!{sys.executable} -m pip install linearmodels

In [40]:
from linearmodels.panel import PanelOLS

### Regression Set Up Regression (Parents and Non-parents- All)

In [50]:
# Sample restriction
df_twfe = df[
    (df['medicaid_eligible'] == 1) &
    (df['sample1'] == 1)
].copy()

# Set panel index
df_twfe = df_twfe.set_index(['statefip', 'year'])

# Create quadratic terms (REQUIRED)
df_twfe['age2'] = df_twfe['age'] ** 2

# Outcome
y = df_twfe['SNAP1']

In [51]:
educ_dummies = pd.get_dummies(
    df_twfe["Educ"],
    prefix="Educ",
    drop_first=True   # lowest education = base
)

In [61]:
X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "female",
                "black",      # white omitted
                "hispanic",
                "otherrace",
                "married",
                "age",
                "age2",
                "famsize",
                "ur",
            ]
        ],
        educ_dummies,        # Educ_2, Educ_3, Educ_4 (drop_first already)
    ],
    axis=1
)

In [62]:
model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    check_rank=False,
    drop_absorbed=True
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
    debiased=False
)

#### Results

In [ ]:
results.summary

# These results are the same as when i used Stata in the estimation

Dep. Variable:,SNAP1,R-squared:,0.0642
Estimator:,PanelOLS,R-squared (Between):,0.1180
No. Observations:,260610,R-squared (Within):,0.0635
Date:,"Mon, Jan 12 2026",R-squared (Overall):,0.1183
Time:,17:22:04,Log-likelihood,-1.638e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1374.4
Entities:,51,P-value,0.0000
Avg Obs:,5110.0,Distribution:,"F(13,260534)"
Min Obs:,2393.0,,
Max Obs:,3.143e+04,F-statistic (robust):,2937.7


### Heterogeneity Analysis

#### By Gender

In [64]:
# REQUIRED TRANSFORMS
df_twfe["age2"] = df_twfe["age"] ** 2
df_twfe["medicaid_female"] = df_twfe["medicaid"] * df_twfe["female"]


educ_dummies = pd.get_dummies(
    df_twfe["Educ"],
    prefix="Educ",
    drop_first=True
)

y = df_twfe["SNAP1"]

X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "female",
                "medicaid_female",
                "white",
                "black",
                "hispanic",
                "otherrace",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,
    ],
    axis=1,
)


model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)
# LINEAR COMBINATION
# lincom 1.medicaid + 1.medicaid#1.female


b = results.params
V = results.cov

L = np.array([1, 0, 1] + [0] * (len(b) - 3))  # medicaid + medicaid_female

beta_lincom = L @ b.values
se_lincom = np.sqrt(L @ V.values @ L.T)
t_lincom = beta_lincom / se_lincom

beta_lincom, se_lincom, t_lincom

(np.float64(0.02247070608090576),
 np.float64(0.011011611834416207),
 np.float64(2.04063732165665))

#### Results

In [57]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0436
Estimator:,PanelOLS,R-squared (Between):,-0.1316
No. Observations:,260610,R-squared (Within):,0.0433
Date:,"Mon, Jan 12 2026",R-squared (Overall):,-0.1072
Time:,17:08:38,Log-likelihood,-1.666e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,848.36
Entities:,51,P-value,0.0000
Avg Obs:,5110.0,Distribution:,"F(14,260533)"
Min Obs:,2393.0,,
Max Obs:,3.143e+04,F-statistic (robust):,171.46


#### By Race (White and Non-white)

In [66]:
df_twfe["medicaid_white"] = df_twfe["medicaid"] * df_twfe["white"]


X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "white",
                "medicaid_white",
                "female",
                "black",
                "hispanic",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,
    ],
    axis=1,
)


model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)


b = results.params
V = results.cov

L = np.zeros(len(b))
L[b.index.get_loc("medicaid")] = 1
L[b.index.get_loc("medicaid_white")] = 1

beta_lincom = L @ b.values
se_lincom = np.sqrt(L @ V.values @ L.T)
t_lincom = beta_lincom / se_lincom

beta_lincom, se_lincom, t_lincom

(np.float64(0.012148126010911482),
 np.float64(0.011400233040777204),
 np.float64(1.0656033054288592))

In [59]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0437
Estimator:,PanelOLS,R-squared (Between):,-0.1207
No. Observations:,260610,R-squared (Within):,0.0433
Date:,"Mon, Jan 12 2026",R-squared (Overall):,-0.0976
Time:,17:15:46,Log-likelihood,-1.666e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,915.70
Entities:,51,P-value,0.0000
Avg Obs:,5110.0,Distribution:,"F(13,260534)"
Min Obs:,2393.0,,
Max Obs:,3.143e+04,F-statistic (robust):,178.14


#### By Education (Highschool completion Status)

In [67]:
# interaction
df_twfe["medicaid_HSorless"] = df_twfe["medicaid"] * df_twfe["HSorless"]

# regressors 
X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "HSorless",
                "medicaid_HSorless",
                "white",
                "black",
                "hispan",
                "female",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,   # same as before (base = no HS)
    ],
    axis=1,
)

# model
model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)

# lincom: 1.medicaid + 1.medicaid#1.HSorless
b = results.params
V = results.cov

L = np.zeros(len(b))
L[b.index.get_loc("medicaid")] = 1
L[b.index.get_loc("medicaid_HSorless")] = 1

beta_lincom = L @ b.values
se_lincom = np.sqrt(L @ V.values @ L.T)
t_lincom = beta_lincom / se_lincom

beta_lincom, se_lincom, t_lincom

(np.float64(0.030047520880216748),
 np.float64(0.010916222082541557),
 np.float64(2.752556759382177))

#### Results

In [68]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0436
Estimator:,PanelOLS,R-squared (Between):,-0.0109
No. Observations:,260610,R-squared (Within):,0.0433
Date:,"Mon, Jan 12 2026",R-squared (Overall):,0.0429
Time:,17:27:50,Log-likelihood,-1.666e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,913.22
Entities:,51,P-value,0.0000
Avg Obs:,5110.0,Distribution:,"F(13,260534)"
Min Obs:,2393.0,,
Max Obs:,3.143e+04,F-statistic (robust):,6.498e+05


#### Regression Set Up and Regression(Non-parents)

In [59]:
# Sample restriction
df_twfe = df[
    (df['medicaid_eligible'] == 1) &
    (df['parent'] == 0) &
    (df['sample1'] == 1)
].copy()

# Set panel index
df_twfe = df_twfe.set_index(['statefip', 'year'])

# Create quadratic terms (REQUIRED)
df_twfe['age2'] = df_twfe['age'] ** 2

# Outcome
y = df_twfe['SNAP1']



In [50]:
educ_dummies = pd.get_dummies(
    df_twfe["Educ"],
    prefix="Educ",
    drop_first=True   # lowest education = base
)


In [43]:
X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "female",
                "black",      # white omitted
                "hispan",
                "otherrace",
                "married",
                "age",
                "age2",
                "famsize",
                "ur",
            ]
        ],
        educ_dummies,        # Educ_2, Educ_3, Educ_4 (drop_first already)
    ],
    axis=1
)

In [49]:
model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    check_rank=False,
    drop_absorbed=True
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
    debiased=False
)

In [53]:
results.params
results.std_errors
results.tstats
results.pvalues

medicaid     2.568988e-03
female       7.825207e-08
black        0.000000e+00
hispan       4.569780e-01
otherrace    2.188563e-01
married      0.000000e+00
age          1.718635e-06
age2         8.012486e-05
famsize      0.000000e+00
ur           8.664517e-01
Educ_2.0     0.000000e+00
Educ_3.0     0.000000e+00
Educ_4.0     0.000000e+00
Name: pvalue, dtype: float64

In [ ]:
results.summary


Dep. Variable:,SNAP1,R-squared:,0.0772
Estimator:,PanelOLS,R-squared (Between):,0.1003
No. Observations:,115670,R-squared (Within):,0.0770
Date:,"Tue, Jan 06 2026",R-squared (Overall):,0.1063
Time:,11:58:45,Log-likelihood,-6.215e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,743.89
Entities:,51,P-value,0.0000
Avg Obs:,2268.0,Distribution:,"F(13,115594)"
Min Obs:,1036.0,,
Max Obs:,1.35e+04,F-statistic (robust):,2338.3


### Heterogeneity Analysis

#### By Gender

In [69]:

# REQUIRED TRANSFORMS
df_twfe["age2"] = df_twfe["age"] ** 2
df_twfe["medicaid_female"] = df_twfe["medicaid"] * df_twfe["female"]


educ_dummies = pd.get_dummies(
    df_twfe["Educ"],
    prefix="Educ",
    drop_first=True
)

y = df_twfe["SNAP1"]

X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "female",
                "medicaid_female",
                "white",
                "black",
                "hispan",
                "otherrace",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,
    ],
    axis=1,
)


model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)
# LINEAR COMBINATION
# lincom 1.medicaid + 1.medicaid#1.female


b = results.params
V = results.cov

L = np.array([1, 0, 1] + [0] * (len(b) - 3))  # medicaid + medicaid_female

beta_lincom = L @ b.values
se_lincom = np.sqrt(L @ V.values @ L.T)
t_lincom = beta_lincom / se_lincom

beta_lincom, se_lincom, t_lincom


(np.float64(0.021597724576706216),
 np.float64(0.00866553593747837),
 np.float64(2.4923703199124985))

In [70]:
results.summary

## The results are the same as what i got in Stata

Dep. Variable:,SNAP1,R-squared:,0.0579
Estimator:,PanelOLS,R-squared (Between):,-0.1656
No. Observations:,115670,R-squared (Within):,0.0575
Date:,"Tue, Jan 06 2026",R-squared (Overall):,-0.1354
Time:,12:54:20,Log-likelihood,-6.334e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,507.36
Entities:,51,P-value,0.0000
Avg Obs:,2268.0,Distribution:,"F(14,115593)"
Min Obs:,1036.0,,
Max Obs:,1.35e+04,F-statistic (robust):,146.98


#### By Race (White and Non-white)

In [73]:


df_twfe["medicaid_white"] = df_twfe["medicaid"] * df_twfe["white"]


X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "white",
                "medicaid_white",
                "female",
                "black",
                "hispan",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,
    ],
    axis=1,
)


model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)


b = results.params
V = results.cov

L = np.zeros(len(b))
L[b.index.get_loc("medicaid")] = 1
L[b.index.get_loc("medicaid_white")] = 1

beta_lincom = L @ b.values
se_lincom = np.sqrt(L @ V.values @ L.T)
t_lincom = beta_lincom / se_lincom

beta_lincom, se_lincom, t_lincom


(np.float64(0.014938106563123492),
 np.float64(0.008970285012975438),
 np.float64(1.665287841079258))

In [74]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0580
Estimator:,PanelOLS,R-squared (Between):,-0.1539
No. Observations:,115670,R-squared (Within):,0.0575
Date:,"Tue, Jan 06 2026",R-squared (Overall):,-0.1251
Time:,13:13:25,Log-likelihood,-6.334e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,547.07
Entities:,51,P-value,0.0000
Avg Obs:,2268.0,Distribution:,"F(13,115594)"
Min Obs:,1036.0,,
Max Obs:,1.35e+04,F-statistic (robust):,139.33


By Education (Highschool graduation status)

In [75]:
# interaction
df_twfe["medicaid_HSorless"] = df_twfe["medicaid"] * df_twfe["HSorless"]

# regressors (match Stata; no nchild, no famsize)
X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "HSorless",
                "medicaid_HSorless",
                "white",
                "black",
                "hispan",
                "female",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,   # same as before (base = no HS)
    ],
    axis=1,
)

# model
model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)

# lincom: 1.medicaid + 1.medicaid#1.HSorless
b = results.params
V = results.cov

L = np.zeros(len(b))
L[b.index.get_loc("medicaid")] = 1
L[b.index.get_loc("medicaid_HSorless")] = 1

beta_lincom = L @ b.values
se_lincom = np.sqrt(L @ V.values @ L.T)
t_lincom = beta_lincom / se_lincom

beta_lincom, se_lincom, t_lincom


(np.float64(0.03353659244227464),
 np.float64(0.008736954820761266),
 np.float64(3.838476120144631))

In [76]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0579
Estimator:,PanelOLS,R-squared (Between):,0.1346
No. Observations:,115670,R-squared (Within):,0.0576
Date:,"Tue, Jan 06 2026",R-squared (Overall):,0.0586
Time:,13:28:41,Log-likelihood,-6.334e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,546.83
Entities:,51,P-value,0.0000
Avg Obs:,2268.0,Distribution:,"F(13,115594)"
Min Obs:,1036.0,,
Max Obs:,1.35e+04,F-statistic (robust):,3.802e+05


#### Estimation for Parents

In [82]:
# Sample restriction
df_twfe = df[
    (df["medicaid_eligible"] == 1) &
    (df["parent"] == 1) &
    (df["sample1"] == 1)
].copy()

# Set panel index
df_twfe = df_twfe.set_index(["statefip", "year"])


# Required transforms
df_twfe["age2"] = df_twfe["age"] ** 2

# Re-create education dummies HERE (important)
educ_dummies = pd.get_dummies(
    df_twfe["Educ"],
    prefix="Educ",
    drop_first=True
)

# Outcome
y = df_twfe["SNAP1"]
# Regressors (match Stata)
X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "female",
                "black",      # white omitted
                "hispan",
                "otherrace",
                "married",
                "age",
                "age2",
                "famsize",
                "ur",
            ]
        ],
        educ_dummies,
    ],
    axis=1
)


In [83]:
model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    check_rank=False,
    drop_absorbed=True
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
    debiased=False
)

In [84]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0425
Estimator:,PanelOLS,R-squared (Between):,-0.1637
No. Observations:,144940,R-squared (Within):,0.0421
Date:,"Tue, Jan 06 2026",R-squared (Overall):,-0.1441
Time:,13:57:19,Log-likelihood,-9.761e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,494.90
Entities:,51,P-value,0.0000
Avg Obs:,2842.0,Distribution:,"F(13,144864)"
Min Obs:,1272.0,,
Max Obs:,1.793e+04,F-statistic (robust):,3184.7


#### Hetergeneity Analysis for parents

#### By Gender

In [ ]:
# interaction
df_twfe["medicaid_female"] = df_twfe["medicaid"] * df_twfe["female"]

# regressors (match Stata)
X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "female",
                "medicaid_female",
                "white",
                "black",
                "hispan",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,   # same as main model (base category already dropped)
    ],
    axis=1,
)

# model
model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)

# lincom: 1.medicaid + 1.medicaid#1.female
b = results.params
V = results.cov

L = np.zeros(len(b))
L[b.index.get_loc("medicaid")] = 1
L[b.index.get_loc("medicaid_female")] = 1

beta_lincom = L @ b.values
se_lincom = np.sqrt(L @ V.values @ L.T)
t_lincom = beta_lincom / se_lincom

from scipy import stats


df_clusters = df_twfe.index.get_level_values("statefip").nunique() - 1

p_lincom = 2 * (1 - stats.t.cdf(abs(t_lincom), df=df_clusters))

beta_lincom, se_lincom, t_lincom, p_lincom





(np.float64(0.02438494774049716),
 np.float64(0.014060487077504749),
 np.float64(1.7342889763406864),
 np.float64(0.08902886494937068))

In [86]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0408
Estimator:,PanelOLS,R-squared (Between):,-0.2532
No. Observations:,144940,R-squared (Within):,0.0406
Date:,"Tue, Jan 06 2026",R-squared (Overall):,-0.2256
Time:,14:08:59,Log-likelihood,-9.773e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,474.34
Entities:,51,P-value,0.0000
Avg Obs:,2842.0,Distribution:,"F(13,144864)"
Min Obs:,1272.0,,
Max Obs:,1.793e+04,F-statistic (robust):,246.60


#### By Race (White and non-white)

In [ ]:
# interaction: Medicaid × White
df_twfe["medicaid_white"] = df_twfe["medicaid"] * df_twfe["white"]


X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "white",
                "medicaid_white",
                "female",
                "black",
                "hispan",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,
    ],
    axis=1,
)

# model
model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)

# lincom: 1.medicaid + 1.medicaid#1.white  (effect for whites)
b = results.params
V = results.cov

L = np.zeros(len(b))
L[b.index.get_loc("medicaid")] = 1
L[b.index.get_loc("medicaid_white")] = 1

beta_white = L @ b.values
se_white = np.sqrt(L @ V.values @ L.T)
t_white = beta_white / se_white


beta_white, se_white, t_white


(np.float64(0.0073106085764143396),
 np.float64(0.016134074193401418),
 np.float64(0.45311608765281763))

In [91]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0410
Estimator:,PanelOLS,R-squared (Between):,-0.2432
No. Observations:,144940,R-squared (Within):,0.0407
Date:,"Tue, Jan 06 2026",R-squared (Overall):,-0.2163
Time:,15:16:19,Log-likelihood,-9.772e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,476.45
Entities:,51,P-value,0.0000
Avg Obs:,2842.0,Distribution:,"F(13,144864)"
Min Obs:,1272.0,,
Max Obs:,1.793e+04,F-statistic (robust):,253.79


#### By Education (Highschool graduation status)

In [92]:
# interaction: Medicaid × HS-or-less
df_twfe["medicaid_HSorless"] = df_twfe["medicaid"] * df_twfe["HSorless"]

# regressors (match Stata; HS-or-less is the heterogeneity group)
X = pd.concat(
    [
        df_twfe[
            [
                "medicaid",
                "HSorless",
                "medicaid_HSorless",
                "white",
                "black",
                "hispan",
                "female",
                "married",
                "age",
                "age2",
                "ur",
            ]
        ],
        educ_dummies,   # base category aligned with Stata
    ],
    axis=1,
)

# model
model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True,
)

results = model.fit(
    cov_type="clustered",
    cluster_entity=True,
)

# lincom: 1.medicaid + 1.medicaid#1.HSorless
b = results.params
V = results.cov

L = np.zeros(len(b))
L[b.index.get_loc("medicaid")] = 1
L[b.index.get_loc("medicaid_HSorless")] = 1

beta_HSorless = L @ b.values
se_HSorless = np.sqrt(L @ V.values @ L.T)
t_HSorless = beta_HSorless / se_HSorless

beta_HSorless, se_HSorless, t_HSorless


(np.float64(0.024325911921066368),
 np.float64(0.0140581246388385),
 np.float64(1.7303810106976123))

In [93]:
results.summary

Dep. Variable:,SNAP1,R-squared:,0.0408
Estimator:,PanelOLS,R-squared (Between):,0.1215
No. Observations:,144940,R-squared (Within):,0.0406
Date:,"Tue, Jan 06 2026",R-squared (Overall):,0.0432
Time:,15:30:59,Log-likelihood,-9.773e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,474.34
Entities:,51,P-value,0.0000
Avg Obs:,2842.0,Distribution:,"F(13,144864)"
Min Obs:,1272.0,,
Max Obs:,1.793e+04,F-statistic (robust):,8.529e+05
